In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
SF = requests.get("http://www.healthysf.org/bdi/outcomes/zipmap.htm")
soup = BeautifulSoup(SF.text, "lxml")
table = soup.find_all("table")
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])
df.columns = df.iloc[0]
# Sort neighborhoods by population
df.sort_values(by=['Population (Census 2000)'], inplace=True)
df = df.iloc[1:-1, :-1]
df_sf = df
df_sf.head()

,Zip Code,Neighborhood
3,94107,Potrero Hill
17,94127,St. Francis Wood/Miraloma/West Portal
15,94123,Marina
2,94103,South of Market
19,94132,Lake Merced


In [5]:
!pip install uszipcode
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in df_sf.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

df_sf["Latitude"] = latitude
df_sf["Longitude"] = longitude

# Drop any null values
df_sf.dropna(inplace=True)
df_sf.head()

     |████████████████████████████████| 378kB 9.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 19.8MB/s eta 0:00:01
     |████████████████████████████████| 122kB 12.3MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/b0/b0/ba/2ab847c5da4ba60ba4682ba33fcc0783c956cda5ff22b2037f
Successfully built autopep8
Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


,Zip Code,Neighborhood,Latitude,Longitude
3,94107,Potrero Hill,37.77,-122.39
17,94127,St. Francis Wood/Miraloma/West Portal,37.73,-122.46
15,94123,Marina,37.80,-122.44
2,94103,South of Market,37.78,-122.41
19,94132,Lake Merced,37.72,-122.48


In [6]:

latitude = 37.7792808
longitude = -122.4192363
sf_map = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_sf['Latitude'], df_sf['Longitude'], df_sf['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(sf_map)  
    
sf_map

NameError: name 'folium' is not defined